In [ ]:
!pip install librosa

In [2]:
import tensorflow as tf
import numpy as np
import librosa
import soundfile as sf
import os
import shutil
from pathlib import Path
import random
from scipy.signal import butter, lfilter
from IPython.display import Audio
     

2025-10-24 05:30:08.652112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761283808.681543     549 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761283808.691374     549 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:

# --- 1. Configuration ---
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})
tf.keras.mixed_precision.set_global_policy('mixed_float16')
input_root_dir="/kaggle/input/milan2025asr/dataset_ASR/data_aug"
output_root_dir = "/kaggle/working/wav_asr"
# ---
Training_dirs=output_root_dir
# Parameters
SAMPLE_RATE = 16000
BATCH_SIZE = 32
EPOCHS = 15 # Set to a higher number for real training

# Create a dummy vocabulary with only uppercase letters and apostrophe
CHARACTERS = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    "'",' ']

# Create character-to-number mappings
char_to_num = tf.keras.layers.StringLookup(vocabulary=list(CHARACTERS), mask_token=None)
num_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
VOCAB_SIZE = char_to_num.vocabulary_size()


I0000 00:00:1761283821.139277     549 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [4]:
# 2. Set the path where you want to save the new dataset (with WAV files)
# code to convert the augmented flac dataset to the new 


print(f"Starting transfer and conversion from '{input_root_dir}'...")
print(f"Output will be saved in '{output_root_dir}'.")

# Walk through the entire directory structure
for dirpath, _, filenames in os.walk(input_root_dir):
    for filename in filenames:
        # Construct the full path to the source file
        input_file_path = os.path.join(dirpath, filename)
        
        # Determine the corresponding output directory path
        relative_path = os.path.relpath(dirpath, input_root_dir)
        output_dir = os.path.join(output_root_dir, relative_path)
        
        # Create the output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)

        # --- Logic to either convert or copy the file ---
        try:
            if filename.lower().endswith(".flac"):
                # It's an audio file, so convert it to WAV
                
                # Create the full path for the output WAV file
                wav_filename = Path(filename).stem + ".wav"
                output_file_path = os.path.join(output_dir, wav_filename)

                # Read the FLAC data and write it as WAV
                # Using soundfile which handles both reading FLAC and writing WAV
                audio_data, sample_rate = sf.read(input_file_path)
                sf.write(output_file_path, audio_data, sample_rate)
                # Optional: Print progress for audio files
                # print(f"Converted: {input_file_path} -> {output_file_path}")

            else:
                # It's a non-audio file (e.g., .txt), so copy it directly
                
                # Construct the output path for the copied file
                output_file_path = os.path.join(output_dir, filename)
                
                # Use shutil.copy2 to preserve file metadata (like modification time)
                shutil.copy2(input_file_path, output_file_path)
                # Optional: Print progress for copied files
                # print(f"Copied: {input_file_path} -> {output_file_path}")

        except Exception as e:
            # Print an error message if any file fails to process
            print(f"Error processing {input_file_path}: {e}")

print("\n--- Process Complete ---")
print(f"New dataset with WAV files is ready at: '{output_root_dir}'")

Starting transfer and conversion from '/kaggle/input/milan2025asr/dataset_ASR/data_aug'...
Output will be saved in '/kaggle/working/wav_asr'.


KeyboardInterrupt: 

In [6]:
# Function to create dummy data
def load_data():
    file_paths = []
    transcriptions = []
    directories=[]
    label_files=[]
    for lfold1 in os.listdir(Training_dirs):
       if True:
        for lfold2 in os.listdir(os.path.join(Training_dirs,lfold1)):
            full_path = os.path.join(Training_dirs, lfold1,lfold2)
            if os.path.isdir(full_path):
                directories.append(full_path)
                label_files.append(os.path.join(Training_dirs, lfold1,lfold2,lfold1+'-'+lfold2+'.trans.txt'))
    for label_path in label_files:
        with open(label_path,'r') as labels:
            for line in labels.readlines():
                transcriptions.append(line.split(' ',maxsplit=1)[1].strip())
    for path in directories:
        fp=[]
        for file in os.listdir(path):
            if(file.endswith('.wav')):
                fp.append(os.path.join(path,file))
        fp.sort()
        file_paths+=fp
    
    print(len(label_files))
        
    return file_paths, transcriptions
#print(os.listdir(Training_dirs))


In [8]:
# --- 2. tf.data Pipeline (No Augmentation) ---
'''
def preprocess_audio(file_path):
    """Loads and converts a FLAC file to a log Mel spectrogram."""
    try:
        path_str = file_path.numpy().decode('utf-8')
        y, sr = librosa.load(path_str, sr=SAMPLE_RATE)
        
        # Compute the Mel spectrogram
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80)
        
        # Convert to log scale (decibels)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        
        # Add a channel dimension
        log_mel_spec = np.expand_dims(log_mel_spec.T, axis=-1)
        
        return log_mel_spec.astype(np.float32)
    except Exception as e:
        print(f"Error processing file {file_path.numpy()}: {e}")
        os.exit()
        return np.zeros((100, 80, 1), dtype=np.float32)
'''       

SAMPLE_RATE = 16000
N_FFT = 400
HOP_LENGTH = 160
N_MELS = 80

def power_to_db(S, ref=1.0, top_db=80.0):
    """Converts a power spectrogram to the decibel scale."""
    log_spec = 10.0 * (tf.math.log(tf.maximum(S, 1e-10)) / tf.math.log(10.0))
    log_spec -= 10.0 * (tf.math.log(tf.maximum(ref, 1e-10)) / tf.math.log(10.0))
    return tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

@tf.function
def preprocess_audio_tf(file_path: tf.Tensor):
    """
    Loads and converts a FLAC file to a log Mel spectrogram using TensorFlow,
    with padding to match librosa's default behavior.
    """
    try:
        
        audio_binary = tf.io.read_file(file_path)
    
        # decode_wav returns a normalized float32 tensor and the sample rate.
        # desired_channels=1 ensures the audio is mono.
        audio_tensor, _ = tf.audio.decode_wav(audio_binary, desired_channels=1)
    
        # Squeeze the channel dimension, leaving a 1D waveform.
        # NO further normalization is needed.
        waveform = tf.squeeze(audio_tensor, axis=-1)

        # --- FIX: Manually pad the waveform to match librosa ---
        # (The rest of your function remains the same and is correct)
        padding = N_FFT // 2
        waveform = tf.pad(waveform, [[padding, padding]], mode="REFLECT")
        
        # --- 2. Compute the STFT (The rest is the same) ---
        stft = tf.signal.stft(
            waveform,
            frame_length=N_FFT,
            frame_step=HOP_LENGTH,
            fft_length=N_FFT
        )
        spectrogram = tf.abs(stft)

        # ... (rest of the function is identical) ...
        power_spectrogram = spectrogram ** 2
        num_spectrogram_bins = stft.shape[-1]
        mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=N_MELS,
            num_spectrogram_bins=num_spectrogram_bins,
            sample_rate=SAMPLE_RATE,
            lower_edge_hertz=20.0,
            upper_edge_hertz=8000.0
        )
        mel_spectrogram = tf.tensordot(power_spectrogram, mel_filterbank, 1)
        log_mel_spectrogram = power_to_db(mel_spectrogram)
        log_mel_spectrogram = tf.expand_dims(log_mel_spectrogram, axis=-1)

        return tf.cast(log_mel_spectrogram, dtype=tf.float32)

    except Exception as e:
        tf.print("Error processing file:", file_path, "Exception:", e, summarize=-1)
        return tf.zeros((100, N_MELS, 1), dtype=tf.float32)
    
def preprocess_label(text_label):
    """Converts a text string to an integer sequence, ensuring it's uppercase."""
    # Convert all characters to uppercase to match the vocabulary
    text_tensor = tf.strings.upper(text_label)
    chars = tf.strings.unicode_split(text_tensor, input_encoding="UTF-8")
    return char_to_num(chars)
# (Keep all your other functions like preprocess_audio_tf_flac, preprocess_label, etc.)

@tf.function
def preprocess_and_filter(path, label):
    """
    Applies full preprocessing to audio and text, and returns their lengths.
    """
    # Process the audio file to get the final spectrogram
    spectrogram = preprocess_audio_tf(path)
    
    # Process the text label to get the integer tokens
    processed_label = preprocess_label(label)

    # Get the number of time steps from the spectrogram
    spectrogram_length = tf.shape(spectrogram)[0]
    
    # Get the number of characters/tokens from the label
    label_length = tf.shape(processed_label)[0]

    return spectrogram, processed_label, spectrogram_length, label_length
#preprocess_audio_tf("/kaggle/working/LibriSpeech-WAV-Complete/1081/125237/1081-125237-0035.wav")

In [9]:
'''def build_pipeline(paths, labels, is_training=False):
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    
    ds = tf.data.Dataset.zip((path_ds, label_ds))
    if is_training:
        ds = ds.shuffle(buffer_size=len(paths))
    
    # Map preprocessing functions
    ds = ds.map(
        lambda path, label: (
            tf.py_function(preprocess_audio, [path], tf.float32),
            preprocess_label(label)
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Batch and pad
    ds = ds.padded_batch(
        batch_size=BATCH_SIZE,
        padded_shapes=([None, 80, 1], [None]),
        padding_values=(0.0, tf.cast(char_to_num.vocabulary_size(), dtype=tf.int64)+1)
    )
    
    # Prefetch for performance
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds'''

def build_pipeline(paths, labels, is_training=False):
    path_ds = tf.data.Dataset.from_tensor_slices(paths)
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    
    ds = tf.data.Dataset.zip((path_ds, label_ds))
    if is_training:
        ds = ds.shuffle(buffer_size=2048)

    # 1. Map the combined preprocessing and length calculation function
    ds = ds.map(preprocess_and_filter, num_parallel_calls=tf.data.AUTOTUNE)
    
    # 2. Filter out items where the spectrogram is shorter than the label
    """ ds = ds.filter(
        lambda spectrogram, label, spec_len, label_len: spec_len >= label_len
    )
    
    # 3. Filter out items where the spectrogram time dimension is too short
    MIN_SPEC_LENGTH=8
    #print(f"Filtering out spectrograms shorter than {MIN_SPEC_LENGTH} time steps.")
   ds = ds.filter(
        lambda spectrogram, label, spec_len, label_len: spec_len >= MIN_SPEC_LENGTH
    )"""
    # 3. Remove the lengths from the dataset, keeping only spectrogram and label
    ds = ds.map(
        lambda spectrogram, label, spec_len, label_len: (spectrogram, label),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # 4. Batch and pad as before
    ds = ds.padded_batch(
        batch_size=BATCH_SIZE,
        padded_shapes=([None, 80, 1], [None]),
        padding_values=(0.0, tf.cast(0, dtype=tf.int64))
    )
    
    # Prefetch for performance
    ds = ds.prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

In [10]:
def build_model(input_shape, vocab_size):
    """Builds a deeper, more regularized CNN-RNN model."""
    inputs = tf.keras.Input(shape=input_shape, name="input_spectrogram")

    # Make the CNN frontend deeper
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x) # <-- Add SpatialDropout
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.SpatialDropout2D(0.2)(x) # <-- Add SpatialDropout
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    # Reshape for the RNN
    _, time_dim, freq_dim, channel_dim = x.shape
    new_feature_dim = freq_dim * channel_dim
    x = tf.keras.layers.Reshape((-1, new_feature_dim))(x)
    
    # Make the RNN backend deeper and with stronger dropout
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.4)(x) # <-- Increased Dropout
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.4)(x) # <-- Increased Dropout
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.4)(x) # <-- Increased Dropout
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True))(x)
    x = tf.keras.layers.Dropout(0.4)(x) # <-- Increased Dropout
    x = tf.keras.layers.BatchNormalization()(x)

    # Output layer
    outputs = tf.keras.layers.Dense(units=vocab_size + 1, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# --- 3. Model Definition and CTC Loss ---

"""older model, has less layers but is proven to underfit given the data."""
'''
def build_model_old1(input_shape, vocab_size):
    """Builds a CNN-RNN model with CTC loss."""
    inputs = tf.keras.Input(shape=input_shape, name="input_spectrogram")

    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    
    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)

    _, time_dim, freq_dim, channel_dim = x.shape
    new_feature_dim = freq_dim * channel_dim
    x = tf.keras.layers.Reshape((time_dim, new_feature_dim))(x)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)
    x = tf.keras.layers.BatchNormalization()(x)

    outputs = tf.keras.layers.Dense(units=vocab_size+1, activation="softmax")(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model
    '''

'\ndef build_model_old1(input_shape, vocab_size):\n    """Builds a CNN-RNN model with CTC loss."""\n    inputs = tf.keras.Input(shape=input_shape, name="input_spectrogram")\n\n    x = tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same")(inputs)\n    x = tf.keras.layers.BatchNormalization()(x)\n    x = tf.keras.layers.MaxPooling2D((2, 2))(x)\n    \n    x = tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)\n    x = tf.keras.layers.BatchNormalization()(x)\n    x = tf.keras.layers.MaxPooling2D((2, 2))(x)\n\n    _, time_dim, freq_dim, channel_dim = x.shape\n    new_feature_dim = freq_dim * channel_dim\n    x = tf.keras.layers.Reshape((time_dim, new_feature_dim))(x)\n    \n    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)\n    x = tf.keras.layers.BatchNormalization()(x)\n    \n    x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True))(x)\n    x = tf.keras.layers.BatchNormalization()(

In [11]:

def ctc_loss(y_true, y_pred):
    batch_len = tf.cast(tf.shape(y_pred)[0], dtype="int64")
    time_steps = tf.cast(tf.shape(y_pred)[1], dtype="int64")

    input_length = time_steps * tf.ones(shape=(batch_len, 1), dtype="int64")
    
    # Compute actual label lengths
    label_length = tf.math.count_nonzero(y_true, axis=1, keepdims=True)
    label_length = tf.cast(label_length, dtype="int64")
    #label_length = tf.minimum(label_length, input_length)
    
    loss = tf.keras.backend.ctc_batch_cost(
        y_true,
        y_pred,
        input_length,
        label_length,
    )


    return loss


In [16]:
# --- 4. Main Training Logic ---

if __name__ == "__main__":
    # load the dataset
    paths, labels = load_data()
    
    # Split data
    split_idx = int(len(paths) * 0.9)
    train_paths, val_paths = paths[:split_idx], paths[split_idx:]
    train_labels, val_labels = labels[:split_idx], labels[split_idx:]
    
    # Build data pipelines
    train_ds = build_pipeline(train_paths, train_labels, is_training=True)
    val_ds = build_pipeline(val_paths, val_labels, is_training=False)
    
    steps_per_epoch = len(train_paths) // BATCH_SIZE
    total_decay_steps = steps_per_epoch * EPOCHS

    #Note:you could use cosine_schedule for the optimizer, which would likely have better convergence over larger epochs , but reducelr is faster.
    #IMP: DO NOT USE BOTH cosine_schedule AND reduce_lr, AS THIS LEADS TO UNPREDICTABLE BEHAVIOUR.
    '''
    cosine_schedule = tf.keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=1e-2,  # The starting learning rate
        decay_steps=total_decay_steps, # The number of steps to decay over
        alpha=0 # The minimum learning rate as a fraction of the initial rate
    )
    Optimizer = tf.keras.optimizers.Adam(learning_rate=cosine_schedule)
    '''
    # Build the model
    # We don't know the exact input shape, so we use None for the time dimension
    model = build_model(input_shape=(None, 80, 1), vocab_size=VOCAB_SIZE)
    model.compile(optimizer="adam", loss=ctc_loss)
    model.summary()
    model = tf.keras.models.load_model("/kaggle/working/kaggle/working/asr_model_best.keras", custom_objects={"ctc_loss": ctc_loss})
    for x_batch, y_batch in train_ds.take(1):
        preds = model(x_batch)
        print("Model output time steps:", preds.shape[1])
        print("Max label length in batch:", tf.reduce_max(tf.math.count_nonzero(y_batch, axis=1)))

    # Set up callbacks
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath="kaggle/working/asr_model_best.keras",
        save_best_only=True,
        monitor="val_loss",
        verbose=1
    )
    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=1,
    min_lr=1e-6,
    verbose=1
    )
    
    # Run some basic checks on the dataset  size before running the model
    print("-" * 20)
    print(f"Number of training file paths: {len(train_paths)}")
    estimated_steps = len(train_paths) // BATCH_SIZE
    print(f"Calculated steps_per_epoch should be around: {estimated_steps}")
    print(f"Checking reported dataset cardinality...")
    reported_cardinality = train_ds.cardinality()
    print(f"train_ds.cardinality() reports: {reported_cardinality.numpy()}")
    print("-" * 20)


    
    print("\n--- Starting Model Training ---")
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        initial_epoch=12,
        callbacks=[model_checkpoint,reduce_lr]
    )
   

585


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_spectrogram (InputLayer)  │ (None, None, 80, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ cast_19 (Cast)                  │ (None, None, 80, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, None, 80, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, None, 80, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, None, 80, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, None, 80, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_8             │ (None, None, 80, 32)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, None, 40, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, None, 40, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, None, 40, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, None, 40, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, None, 40, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout2d_9             │ (None, None, 40, 64)   │             0 │
│ (SpatialDropout2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, None, 20, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_4 (Reshape)             │ (None, None, 1280)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_16                │ (None, None, 512)      │     3,147,776 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, None, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_17                │ (None, None, 512)      │     1,574,912 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, None, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 7,961,854 (30.37 MB)

 Trainable params: 7,957,374 (30.35 MB)

 Non-trainable params: 4,480 (17.50 KB)

Model output time steps: 402
Max label length in batch: tf.Tensor(260, shape=(), dtype=int64)
--------------------
Number of training file paths: 25685
Calculated steps_per_epoch should be around: 802
Checking reported dataset cardinality...
train_ds.cardinality() reports: 803
--------------------

--- Starting Model Training ---
Epoch 13/15
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - loss: 110.3660
Epoch 13: val_loss improved from inf to 110.35689, saving model to kaggle/working/asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 1325s 2s/step - loss: 110.3648 - val_loss: 110.3569 - learning_rate: 5.0000e-04
Epoch 14/15
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 797ms/step - loss: 105.9582
Epoch 14: val_loss improved from 110.35689 to 109.17303, saving model to kaggle/working/asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 671s 836ms/step - loss: 105.9585 - val_loss: 109.1730 - learning_rate: 5.0000e-04
Epoch 15/15
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 777ms/step - loss: 105.5354
Epoch 15: val_loss impr

In [17]:

    # Save the final model
    model.save("/kaggle/working/asr_model_final_ep15.keras")
    print("\n--- Training Part 1 complete. Final model saved as asr_model_final_ep15.keras ---")
    print("Best performing model during training saved as asr_model_best.keras")

In [24]:
# Load model from checkpoint /kaggle/input/asr-midtrained/tensorflow2/default/1/asr_model_final_ep15.keras
model = tf.keras.models.load_model("/kaggle/working/asr_model_best.keras", custom_objects={"ctc_loss": ctc_loss})
model.optimizer.learning_rate.assign(1e-5)

# Recreate the same callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="asr_model_best.keras",
    save_best_only=True,
    monitor="val_loss",
    verbose=1
)

# Resume training from epoch 15 → 25
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    initial_epoch=15,
    callbacks=[model_checkpoint, reduce_lr]
)

model.save("/kaggle/working/asr_model_final_ep25.keras")
print("\n--- Training Part 2 complete. Final model saved as asr_model_final_ep15.keras ---")
print("Best performing model during training saved as asr_model_best.keras")

Epoch 16/25
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 769ms/step - loss: 99.4657
Epoch 16: val_loss improved from inf to 104.98033, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 658s 810ms/step - loss: 99.4642 - val_loss: 104.9803 - learning_rate: 1.0000e-05
Epoch 17/25
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - loss: 98.5005
Epoch 17: val_loss improved from 104.98033 to 104.60508, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 647s 806ms/step - loss: 98.4992 - val_loss: 104.6051 - learning_rate: 1.0000e-05
Epoch 18/25
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 771ms/step - loss: 97.6072
Epoch 18: val_loss improved from 104.60508 to 104.36993, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 650s 810ms/step - loss: 97.6063 - val_loss: 104.3699 - learning_rate: 1.0000e-05
Epoch 19/25
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 761ms/step - loss: 97.1505
Epoch 19: val_loss improved from 104.36993 to 104.04418, saving model to asr_model_best.keras
803/803 ━━━━━━━━

In [25]:

model = tf.keras.models.load_model("/kaggle/working/asr_model_best.keras", custom_objects={"ctc_loss": ctc_loss})
model.optimizer.learning_rate.assign(1e-3)

# Recreate the same callbacks
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-7,
    verbose=1
)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="asr_model_best.keras",
    save_best_only=True,
    monitor="val_loss",
    verbose=1
)

# Resume training from epoch 25 → 35
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=35,
    initial_epoch=25,
    callbacks=[model_checkpoint, reduce_lr]
)

model.save("/kaggle/working/asr_model_final_ep35.keras")
print("\n--- Training Part 3 complete. Final model saved as asr_model_final_ep15.keras ---")
print("Best performing model during training saved as asr_model_best.keras")

Epoch 26/35
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 761ms/step - loss: 106.4837
Epoch 26: val_loss improved from inf to 113.00412, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 651s 801ms/step - loss: 106.4878 - val_loss: 113.0041 - learning_rate: 0.0010
Epoch 27/35
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 765ms/step - loss: 108.2911
Epoch 27: val_loss improved from 113.00412 to 112.75507, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 646s 804ms/step - loss: 108.2913 - val_loss: 112.7551 - learning_rate: 0.0010
Epoch 28/35
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 762ms/step - loss: 106.2166
Epoch 28: val_loss improved from 112.75507 to 108.78432, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━━━━━━━━ 643s 801ms/step - loss: 106.2176 - val_loss: 108.7843 - learning_rate: 0.0010
Epoch 29/35
803/803 ━━━━━━━━━━━━━━━━━━━━ 0s 760ms/step - loss: 103.8715
Epoch 29: val_loss improved from 108.78432 to 108.54433, saving model to asr_model_best.keras
803/803 ━━━━━━━━━━━━━

KeyboardInterrupt: 